# Creating Output Rasters

The following script converts Raven model output into raster files readily accessible to standard GIS software (e.g., ArcGIS, QGIS). In addition, simulated long-term average recharge rates are converted into MODFLOW-ready files to simulate the groundwater system (in steady-state) under the Raven-simulated recharge regime.

## Load Python packages and import data

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import shapefile
from PIL import Image, ImageDraw

### Set model output directory, user settings

> Users must change the following input according to the model they wish to convert.

In [2]:
indir = '../model_baseline/output/' # directory of Raven output
modflowdir = '../MODFLOW/'
hrufp = '../model_baseline/Raven2025-hruid.bil' # HRU reference map, produced using update_landuse.ipynb
swsfp = '../GIS/CH_Subwatershed_select.shp' # Raven sub-basin polygons
mdlprfx = 'Raven2025' # model short-name prefix

grid definition

In [3]:
# (keep consistent with input raster definition)
nrows = 719
ncols = 726
xul = 1323330
yul = 11906070
cs = 60. # grid cell width

Raven to MODFLOW mapping

In [4]:
mapfp = '../scripts/supp/Raven2025_to_SHModel-CH_310_map.pkl' # used to map Raven HRUs to MODFLOW cells

## Import HRU ID raster

In [ ]:
# Reads integer rasters
def readIntRaster(fp):
    aa = np.fromfile(fp,np.int32)
    if len(aa) == nrows*ncols:
        x = dict(zip(np.arange(nrows*ncols),aa))
    elif len(aa) == nrows*ncols/2:
        aa = np.fromfile(fp,np.int16)
        x = dict(zip(np.arange(nrows*ncols),aa))
    else:
        print(" ** Warning, unknown raster size {}",fp)
        return None
    return x

In [ ]:
xhru = readIntRaster(hrufp) # hruid to MODFLOW cell cross-reference

## Import Sub-Basins

- Collect raster cells contained within each sub-basin

In [ ]:
def polygonToCellIDs(pts, nrows=719, ncols=726, xul=1323330, yul=11906070, cs=60):
    img = Image.new('L', (ncols, nrows), 0)
    offset = [xul, yul-nrows*cs]
    pixelpoly = [((x-offset[0])/cs, nrows-(y-offset[1])/cs) for x,y in pts]
    ImageDraw.Draw(img).polygon(pixelpoly, outline=1, fill=1)
    mask = np.array(img)
    return np.arange(nrows*ncols)[mask.reshape(nrows*ncols)>0]    

def loadSWS(fp):
    sf = shapefile.Reader(fp)
    geom = sf.shapes()
    attr = sf.records()
    xr = dict()
    for i in range(len(geom)):
        sid = int(attr[i].SubId)
        xr[sid] = polygonToCellIDs(geom[i].points)
    return xr

In [ ]:
xsb = loadSWS(swsfp) # sub-basin to cell cross reference

## Import Model Outputs

- Precipitation
- Total actual evapotranspiration (AET)
- Runoff:
    - from impervious areas
    - hortonian runoff (form pervious areas)
    - delayed runoff
    - interflow
- Groundwater recharge
- Infiltration
- Baseflow

### Model output file read function

In [ ]:
def readRaven(fp, isaccumulated, warmup=24):
    # load data
    df = pd.read_csv(fp, skiprows=1, parse_dates=['month'], date_format="%Y-%m")
    df = df.iloc[warmup:, :-1] # drop warmup period and drop last column (..a Raven quirk)

    df['month'] = pd.to_datetime(df['month'])
    df['month'] = df['month'].dt.month

    # drop columns
    df = df.drop(columns='time')
        
    # rename columns
    cols = [w.replace('mean.','').replace('cumulsum.','') for w in list(df.columns)]
    cols[1] = '0'
    df.columns = cols

    # change accumulation to discrete monthly values
    if isaccumulated:
        df2 = df.diff()
        df2 = df2.fillna(df)
        df2['month'] = df['month']
        df = df2

    df = df.groupby(['month']).mean().sum() # long-term mean
    return {int(k)+1:v for k,v in df.items()}


In [ ]:
dprecip = readRaven(indir+mdlprfx+'_PRECIP_Monthly_CumulSum_ByHRU.csv',False)                                 # precipitation
daet = readRaven(indir+mdlprfx+'_AET_Monthly_CumulSum_ByHRU.csv',False)                                       # evapotranspiration
dimpro = readRaven(indir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)    # impervious runoff
dprvro = readRaven(indir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_CONVOLUTION[0]_Monthly_Average_ByHRU.csv',True)   # pervious/hortonion runoff
ddlyro = readRaven(indir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_CONVOLUTION[1]_Monthly_Average_ByHRU.csv',True)   # delayed runoff
dintflw = readRaven(indir+mdlprfx+'_BETWEEN_SOIL[0]_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)        # interflow/hypodermic flkow
drecharge = readRaven(indir+mdlprfx+'_TO_SOIL[1]_Monthly_Average_ByHRU.csv',True)                             # recharge
dinfilt =  readRaven(indir+mdlprfx+'_BETWEEN_PONDED_WATER_AND_SOIL[0]_Monthly_Average_ByHRU.csv',True)        # infiltration
dbasflw = readRaven(indir+mdlprfx+'_BETWEEN_SOIL[1]_AND_SURFACE_WATER_Monthly_Average_ByHRU.csv',True)        # baseflow

## Map Model Outputs to Raster Files

In [ ]:
def dictToNp(dat, nr, nc, nodata=-9999.):
    def crc(c): # cell to row-col
        i = int(c/nc)
        j = c - i*nc
        return (i,j)    

    a = np.full((nr,nc),nodata,dtype=np.float32)
    for cid,v in dat.items(): a[crc(cid)] = np.float32(v)
    return a


def saveBinaryFloat(fp, dat):
    fn, _ = os.path.splitext(fp)
    with open(fn+'.hdr', 'w') as f: # save header
        f.write('BYTEORDER      I\n')
        f.write('LAYOUT         BIL\n')
        f.write('NROWS          '+str(nrows)+'\n')
        f.write('NCOLS          '+str(ncols)+'\n')
        f.write('NBANDS         1\n')
        f.write('NBITS          32\n')
        f.write('BANDROWBYTES   '+str(nrows*4)+'\n')
        f.write('TOTALROWBYTES  '+str(nrows*4)+'\n')
        f.write('PIXELTYPE      FLOAT\n')
        f.write('ULXMAP         '+str(xul+cs/2)+'\n') # The x-axis map coordinate of the center of the upper-left pixel.
        f.write('ULYMAP         '+str(yul-cs/2)+'\n') # The y-axis map coordinate of the center of the upper-left pixel.
        f.write('XDIM           '+str(cs)+'\n')
        f.write('YDIM           '+str(cs)+'\n')
        f.write('NODATA         -9999\n')

    a = dictToNp(dat,nrows, ncols)
    if os.path.exists(fp): os.remove(fp)
    a.tofile(fp) # always saved in C-order (row-major)    

Water Balance

$$ P = E + R + G + \Delta S $$

where $P$ is precipitation, $E$ is total evapotranspiration, $R$ is runoff, $G$ is groundwater recharge, and $\Delta S$ is the change in surface storage, calculated as the residual.

In [ ]:
pre = dict()
aet = dict()
roff = dict()
rch = dict()
rsid = dict()
for c,h in xhru.items():
    if h==-9999: continue
    pre[c]=dprecip[h]
    aet[c]=daet[h]
    roff[c]=dimpro[h]+dprvro[h]+ddlyro[h]+dintflw[h]
    rch[c]=drecharge[h]
    rsid[c]=pre[c]-aet[c]-roff[c]-rch[c]

saveBinaryFloat(indir+mdlprfx+'_longterm.precipitation.bil',pre)
saveBinaryFloat(indir+mdlprfx+'_longterm.evapotranspiration.bil',aet)
saveBinaryFloat(indir+mdlprfx+'_longterm.runoff.bil',roff)
saveBinaryFloat(indir+mdlprfx+'_longterm.recharge.bil',rch)
saveBinaryFloat(indir+mdlprfx+'_longterm.residual.bil',rsid)

## Create MODFLOW RCH file

The remaining code block uses only the groundwater recharge estimates to produce:

1. MODFLOW recharge input file (_*RCH_) 
2. Significant Groundwater Recharge Areas (SGRAs) exported as a raster

First, defined MODFLOW grid definition of `SHModel-CH_310-NWT`:

In [ ]:
gwNrow = 885
gwNcol = 890

In [ ]:
with open(mapfp,'rb') as f: gmap = pickle.load(f)
grch, wrch = dict(), dict()
for c,ws in  gmap.items():
    for cRaven,w in ws.items():
        if cRaven==-1: continue
        if c in grch:
            grch[c]+=w*rch[cRaven]
            wrch[c]+=w
        else:
            grch[c]=w*rch[cRaven]
            wrch[c]=w

meanrch = sum(grch.values()) / len(grch) 
for c in gmap:
    if c in grch: 
        grch[c]/=wrch[c]
    else:
        grch[c]=meanrch

arch = dictToNp(grch, gwNrow, gwNcol, 0)
arch.tofile(modflowdir+mdlprfx+ ".rch.bil")
hdr = '         3        49   NRCHOP,IRCHCB\n    0    0   INRECH,INIRCH\nINTERNAL     3.1689E-11 (890E11.3)   -1   Longterm groundwater recharge from {}'.format(mdlprfx)
np.savetxt(modflowdir+mdlprfx+ ".rch", arch, header=hdr, comments='', fmt='%10.3e', delimiter=' ')


## Build SGRAs

In [ ]:
# mean = np.mean(rch.values)
sgra = np.array(list(rch.values()))
rchavg = sgra.mean()
print(rchavg) # mean recharge
sgra = sgra >= rchavg * 1.15
saveBinaryFloat(indir+mdlprfx+'_longterm.recharge-SGRA.bil',rch)